In [1]:
import tensorflow, pandas as pd, numpy as np, matplotlib as plt, os, math

In [39]:
def find(name, path):
    for root, dirs, files in os.listdir(path):
        if name in files:
            return os.path.join(root, name)

In [ ]:
os.listdir(path)

In [56]:
print(find("Lord-Saunders", "./Graphs/Building"))

None


In [52]:
path = "./Graphs/Building/"
def find(name, path):
    for file in os.listdir(path):
        if name in file:
            return path + file
find("Adam", path)

'./Graphs/Building/Adam Joseph Lewis Center - Gross Electricity Consumption (kWh)_Electrical.png'

In [57]:
os.listdir(path)

['Adam Joseph Lewis Center - Gross Electricity Consumption (kWh)_Electrical.png',
 'Asia House - Residential Electricity Use (kWh)_Electrical.png',
 'Noah - Electricity Use (kWh)_Electrical.png',
 'Harvey - Electricity Use (kWh)_Electrical.png',
 'Kohl Building - Total Electricity Use (kWh)_Electrical.png',
 'Lord-saunders Electricity use (kWh)_Electrical.png',
 'Fairchild - Residential Electricity Use (kWh)_Electrical.png',
 'North - Electricity Use (kWh)_Electrical.png',
 '.ipynb_checkpoints',
 'Talcott - Residential Electricity Use (kWh)_Electrical.png',
 'Kahn - Total Electricity Use (kWh)_Electrical.png',
 'Science Center - Electricity Use (kWh)_Electrical.png',
 'Noah - Electricity Use (kWh).1_Electrical.png',
 'East - Electricity Use (kWh)_Electrical.png']

In [ ]:
    data = pd.DataFrame()
    path = "Data/Raw Data/"
    for file in os.listdir(path):
      data = pd.concat((data, pd.read_csv(path+file, skiprows=3)), ignore_index = True)

    data["Timestamp"] = pd.to_datetime(data['Timestamp'])
    data.sort_values("Timestamp", ignore_index=True, inplace=True)

    #combine lord saunder
    data["Lord-Saunders Electricity use (kWh)"] = data["Lord - Electricity Use (kWh)"] + data["Saunders - Electricity Use (kWh)"]
    data.drop(["Lord - Electricity Use (kWh)", "Saunders - Electricity Use (kWh)"], axis = 1, inplace = True)
    data.to_csv("Data/Electrical_All.csv", index=False)   #save datas to csv

    # re-format
    data.set_index('Timestamp', inplace=True)
    data = data[data.columns[[0,6,9,1,2,5,7,8,3,4,7,10,11]]]  # re-arrange to building types
    data = data.groupby(data.index.floor('d')).mean()    # average electrical usage by days

    data = remove_outliers(data, 50)   # remove huge outliers
    data.to_csv("Data/Electrical_Averaged.csv")   # save processed data